In [1]:
import pandas as pd

actual_df = pd.read_csv('./evaluation_test/test_labels.csv.zip')
predictions_df = pd.read_csv('./evaluation_test/tox_comment_predictions_v2.csv')

In [2]:
actual_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [3]:
predictions_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.996218,0.670566,0.988770,0.785564,0.968999,0.415051
1,0000247867823ef7,0.000048,0.000002,0.000065,0.000007,0.000033,0.000010
2,00013b17ad220c46,0.000115,0.000002,0.000056,0.000008,0.000028,0.000012
3,00017563c3f7919a,0.000064,0.000004,0.000042,0.000020,0.000043,0.000016
4,00017695ad8997eb,0.000106,0.000002,0.000034,0.000007,0.000026,0.000010


In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
import string
import torchmetrics

from transformers import pipeline
from tqdm.notebook import tqdm
tqdm.pandas()

In [5]:
%pip install -q torchmetrics watermark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%load_ext watermark

In [7]:
%watermark --iversions

torchmetrics: 0.11.4
pandas      : 1.4.3
transformers: 4.25.1
numpy       : 1.23.5
torch       : 2.0.0
sklearn     : 1.1.2



In [8]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [9]:
actual_df_final = actual_df[(actual_df['toxic'] != -1) & (actual_df['severe_toxic'] != -1) & (actual_df['obscene'] != -1) & (actual_df['threat'] != -1) & (actual_df['insult'] != -1) & (actual_df['identity_hate'] != -1)]
actual_df_final

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0,0,0,0,0,0
153151,fff9d70fe0722906,0,0,0,0,0,0
153154,fffa8a11c4378854,0,0,0,0,0,0
153155,fffac2a094c8e0e2,1,0,1,0,1,0


In [10]:
predictions_df_final = predictions_df[predictions_df['id'].isin(actual_df_final['id'])]
predictions_df_final

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0.000065,0.000006,0.000060,0.000031,0.000061,0.000028
7,000247e83dcc1211,0.670541,0.000195,0.001439,0.000404,0.000656,0.000126
11,0002f87b16116a7f,0.044633,0.000320,0.035768,0.000855,0.005448,0.000854
13,0003e1cccfd5a40a,0.000046,0.000003,0.000050,0.000009,0.000025,0.000012
14,00059ace3e3e9a53,0.000052,0.000004,0.000055,0.000016,0.000029,0.000016
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0.000052,0.000004,0.000075,0.000016,0.000039,0.000019
153151,fff9d70fe0722906,0.995920,0.016160,0.967160,0.000610,0.857397,0.000391
153154,fffa8a11c4378854,0.988090,0.007495,0.016223,0.009943,0.121335,0.274623
153155,fffac2a094c8e0e2,0.994882,0.561604,0.988041,0.036383,0.974018,0.973960


### AUC ROC

In [11]:
# Score for each cat

for cat in label_columns:
    
    print(f"Category: {cat}")
    print(f"Sklearn score: {metrics.roc_auc_score(actual_df_final[cat], predictions_df_final[cat])}")
    print(f"torchmetrics score: {torchmetrics.functional.classification.binary_auroc(torch.tensor(predictions_df_final[cat].values),torch.tensor(actual_df_final[cat].values), thresholds=None)}")
    print("#" * 30)
    print()

Category: toxic
Sklearn score: 0.9689133285293112
torchmetrics score: 0.9689133167266846
##############################

Category: severe_toxic
Sklearn score: 0.9903705196910187
torchmetrics score: 0.9903705716133118
##############################

Category: obscene
Sklearn score: 0.9802577184793353
torchmetrics score: 0.9802576899528503
##############################

Category: threat
Sklearn score: 0.9948682767394358
torchmetrics score: 0.994868278503418
##############################

Category: insult
Sklearn score: 0.9781428766814926
torchmetrics score: 0.9781429767608643
##############################

Category: identity_hate
Sklearn score: 0.9880265444243443
torchmetrics score: 0.98802649974823
##############################



In [12]:
print(f"Sklearn score: {metrics.roc_auc_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values)}")
print(F"torchmetrics score: {torchmetrics.functional.classification.multilabel_auroc(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None)}")

print()
print(f"Sklearn score (weighted): {metrics.roc_auc_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values, average='weighted')}")
print(F"torchmetrics score (weighted): {torchmetrics.functional.classification.multilabel_auroc(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None,average='weighted' )}")

Sklearn score: 0.9834298774241562
torchmetrics score: 0.9834299087524414

Sklearn score (weighted): 0.9758426772511357
torchmetrics score (weighted): 0.9758426547050476


### Average Precision

In [13]:
# Score for each cat

for cat in label_columns:
    
    print(f"Category: {cat}")
    print(f"Sklearn score: {metrics.average_precision_score(actual_df_final[cat], predictions_df_final[cat])}")
    print(f"torchmetrics score: {torchmetrics.functional.classification.binary_average_precision(torch.tensor(predictions_df_final[cat].values),torch.tensor(actual_df_final[cat].values), thresholds=None)}")
    print("#" * 30)
    print()

Category: toxic
Sklearn score: 0.764503889153176
torchmetrics score: 0.7645038962364197
##############################

Category: severe_toxic
Sklearn score: 0.35199113456228776
torchmetrics score: 0.35199111700057983
##############################

Category: obscene
Sklearn score: 0.7900971141109929
torchmetrics score: 0.7900971174240112
##############################

Category: threat
Sklearn score: 0.571931829261402
torchmetrics score: 0.571931779384613
##############################

Category: insult
Sklearn score: 0.7582821410261534
torchmetrics score: 0.7582821249961853
##############################

Category: identity_hate
Sklearn score: 0.6308036202658449
torchmetrics score: 0.6308035850524902
##############################



In [14]:
print(f"Sklearn score: {metrics.average_precision_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values)}")
print(F"torchmetrics score: {torchmetrics.functional.classification.multilabel_average_precision(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None)}")

print()
print(f"Sklearn score (weighted): {metrics.average_precision_score(actual_df_final[label_columns].values, predictions_df_final[label_columns].values, average='weighted')}")
print(F"torchmetrics score (weighted): {torchmetrics.functional.classification.multilabel_average_precision(torch.tensor(predictions_df_final[label_columns].values),torch.tensor(actual_df_final[label_columns].values),num_labels=6,thresholds=None,average='weighted' )}")

Sklearn score: 0.6446016213966428
torchmetrics score: 0.6446016430854797

Sklearn score (weighted): 0.7497379342261651
torchmetrics score (weighted): 0.7497379183769226
